# Crawl CNA news from several categories

Only 5 pieces of news on the first page from each category

中央社新聞各類新聞各5篇


In [1]:
import requests
from bs4 import BeautifulSoup
import re

# There are 11 categories in CNA news

Crawl links on a category's first page.

中央社新聞爬蟲-11類別的新聞(只能爬第一頁的內容，沒有翻頁的功能)

In [2]:
!pip install fake_useragent

## Crawl news for 11 categories

In [3]:
# 11 news categories
#news_links =['aipl', 'ait', 'aspt', 'asc', 'aie', 'amov','ahel','aopl','asoc','acul','acn']
#news_categories=['政治','科技','運動','證卷','產經','娛樂','生活','國際','社會','文化','兩岸']
news_links =['aipl', 'ait', 'aspt']
news_categories=['政治','科技','運動']
base_url = 'https://www.cna.com.tw/list/'

In [4]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta
from fake_useragent import UserAgent

In [5]:
user_agent = UserAgent()

In [ ]:
%%time
# 存放資料之變數
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []


# 這裡我們需要用for迴圈爬取11個類別
# Here we need to crawl 11 categories using for loop
for i, url_short_name in enumerate(news_links):  #針對每一類 共有11類

    category = news_categories[i] #'政治' #類別名稱紀錄起來 

    # Categorical url link
    category_url = base_url + url_short_name + '.aspx'
    print("Getting categorical news:", category)

    # Request the categorical news page
    # req = requests.get(category_url)
    req = requests.get(category_url, headers={ 'user-agent': user_agent.random }, timeout=5)
    page = BeautifulSoup(req.text, 'lxml')
    print(page)

    # 抓新聞列表
    items = page.find('ul', {'id': re.compile("jsMainList\w*")}).findAll('li')
    print(items)


    # Let's start to crawl the news in the first page for that category
    serial_no = 1
    for item_j, item in enumerate(items,start=1): #針對每一篇項目 抓其細節
        title = item.find('h2').text
        print(serial_no,'--', title )
        
        link = item.find('a').get('href')
        print(link)

        try:
            if item.find('img').has_attr('data-src'):
                photo_link = item.find('img').get('data-src')
            else:
                photo_link = item.find('img').get('src')
        except:
            photo_link = ''
        
        # get date and convert it with the format of  "2022-04-20"
        news_date = item.find('div', {'class':"date"}).text
        # print(news_date)
        dtime = datetime.strptime(news_date, '%Y/%m/%d %H:%M')
        # conver to the format of  "2022-04-20"
        news_date_str = dtime.strftime("%Y-%m-%d") # keep only date, remove the time.

        categories.append(category)
        titles.append(title)
        links.append(link)
        photo_links.append(photo_link)
        dates.append(news_date_str)
 
        # item_id.append("cna_" + tstr + "_" + str(serial_no))
        # conver to the format of  "20220420"
        dtstr = dtime.strftime("%Y%m%d") # keep only date, remove the time.
        # format of serial number: aipl_20220314_1
        
        item_id.append(url_short_name + "_" + dtstr + "_" + str(serial_no))
        serial_no += 1

        #抓新聞內容
        # 最新的新聞列表之新聞連結:開頭缺少"https://www.cna.com.tw"
        if link.startswith("/news"):
            link = "https://www.cna.com.tw"+link
        
        print(link)
        req = requests.get(link, headers={ 'user-agent': user_agent.random }, timeout=5)
        
        page = BeautifulSoup(req.text,'lxml')

        # Find content
        content = page.find('div',{'class':"paragraph"})

        # Remove unnecessary content
        # There are some words we don't like to analyze, They should be removed.
        # Remove "透過 Google News追蹤中央社"
        try:
            content.find('div', {'class':"gmailNews"}).decompose()
        except:
            pass
        # 額外插入的延伸閱讀區塊-->移除之
        try:
            content.find('div', {'class':"paragraph moreArticle"}).decompose()
        except:
            pass
        # 額外插入的訂閱連結-->移除之
        try:
            content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
        except:
            pass

        # 額外插入的圖片與文字說明區塊-->移除之
        try:
            for media in content.findAll('div', {'class':"media"}):
                media.decompose()
        except:
            pass

        # Remove unnecessary content
        cont = content.text
        cont = re.sub('（中央社.+?）','', cont)
        cont = re.sub('（編輯：.*','', cont) 
        cont = re.sub('（譯者：.*','', cont) 
        
        contents.append(cont)

        if item_j >= 4: # Here we crawl only 4 pieces of news for each category, in order to save time.
            break # 跳開一層迴圈
        
        time.sleep(10)  # 遵守爬蟲禮節，請小睡一下


Getting categorical news: 政治
[<li><a href="/news/aipl/202503120373.aspx"><div class="wrap"><img alt="國合會助貝里斯開創香草產業 勉小農婦女經濟自主" class="lazyload" data-src="https://imgcdn.cna.com.tw/www/WebPhotos/200/20250312/1365x768_wmkn_0_C20250312000243.jpg"/></div><div class="listInfo"><h2><span>國合會助貝里斯開創香草產業 勉小農婦女經濟自主</span></h2><div class="date">2025/03/12 20:40</div></div></a></li>, <li><a href="/news/aipl/202503120363.aspx"><div class="wrap"><img alt="國會衝突綠委控傷害 藍白立委收到檢方傳票" class="lazyload" data-src="https://imgcdn.cna.com.tw/www/webphotos/WebCover/420/20250312/1838x1378_836766746345.jpg"/></div><div class="listInfo"><h2><span>國會衝突綠委控傷害 藍白立委收到檢方傳票</span></h2><div class="date">2025/03/12 19:49</div></div></a></li>, <li><a href="/news/aipl/202503120360.aspx"><div class="wrap"><img alt="總預算財劃法覆議被封殺 吳思瑤：創3個前所未見災害" class="lazyload" data-src="https://imgcdn.cna.com.tw/www/WebPhotos/200/20250312/1152x768_wmky_0_C20250312000173.jpg"/></div><div class="listInfo"><h2><span>總預算財劃法覆議被封殺 吳思瑤：創3個前所未見災害</span></

In [6]:
titles

['國合會助貝里斯開創香草產業 勉小農婦女經濟自主',
 '國會衝突綠委控傷害 藍白立委收到檢方傳票',
 '總預算財劃法覆議被封殺 吳思瑤：創3個前所未見災害',
 '民進黨：藍白否決覆議案  歷史罪人勢必受民意制裁',
 '印尼學者：盼台積電助攻半導體發展 根留台灣最重要',
 'NASA太空望遠鏡SPHEREx發射升空 探索宇宙起源[影]',
 '新漢參展輝達GTC大會 將發表AI機器人控制器',
 '全球首創 國研院打造互動式實驗動物顯微手術基地',
 'U18亞洲盃5人制棒球鐘家樂攻守俱佳 也是氣氛組',
 'BWF全英公開賽 李佳豪化解6個賽末點晉16強',
 '關達祐首披國家隊戰袍像夢一樣 期待與陳盈駿聯手',
 'U18亞洲盃5人制棒球台灣勝日 4勝晉冠軍戰機會大']

In [7]:
contents

['國合會今天表示，駐貝里斯技術團開班教種香草，30名小農及婦女日前順利結訓，當地官員期勉提升經濟自主；國合會副秘書長謝佩芬則說，香草在貝里斯尚未發展商業化生產，將持續協助當地產業發展。國合會今天發布新聞稿指出，駐貝里斯技術團於6日舉辦「小農香草種植培訓班」結訓典禮，本次培訓為國合會「後疫情時期協助拉丁美洲及加勒比海經濟復甦暨婦女賦權計畫」的重要一環，共有30名學員順利結訓，涵蓋加勒比海農業研究與發展機構（CARDI）人員、農業專業高中師生及社區婦女。國合會指出，課程結束後，每位婦女學員皆獲贈10株香草苗在自家後院實踐所學，開啟小規模生產。來自機構及學校的代表則將帶回香草苗，分別於所屬農場栽種，與兩座區域育苗中心共同發揮示範推廣的作用。駐貝里斯大使徐儷文說，對台灣在貝國香草產業發展初期即提供實質協助感到欣慰。國合會副秘書長謝佩芬表示，香草是昂貴的香料，在食品、化妝品與工業領域需求旺盛，市場前景廣闊，貝里斯具備理想生長環境，卻尚未發展商業化生產，樂見技術團發掘這項契機，未來將持續協助貝里斯投入香草產業發展。',
 '去年底立法院審查選罷法等案，朝野立委在議場爆發肢體衝突，民進黨立委對國民黨、民眾黨立委提告傷害。國民黨立委陳玉珍、徐巧芯、翁曉玲及民眾黨立委林憶君等人近日收到傳票，陳玉珍表示，當時只是基於議事攻防，綠委的行為是浪費司法資源。立法院去年底審議選罷法、憲訴法及財劃法三大法案時，多名朝野立委因激烈肢體衝突而受傷。多名民進黨女立委相繼指控遭國民黨立委陳玉珍攻擊。民進黨立委范雲、林宜瑾和陳培瑜等5人認為陳玉珍在議場內蓄意傷人，到北院提告傷害罪。知情人士今天指出，除陳玉珍外，國民黨立委徐巧芯、邱若華、翁曉玲，以及民眾黨立委林憶君等人，也被林宜瑾和陳培瑜提告傷害，國民黨立委黃仁近日也收到傳票。陳玉珍接受中央社記者訪問時，證實收到傳票。她表示，去年底，國民黨依慣例在議場前排班，希望處理選罷法、憲訴法及財劃法的修法，結果前一晚，民進黨立委卻先破窗、破壞公物，占領議場。隔天為讓議事順利進行，國民黨立委對占領主席台的民進黨立委進行排除。陳玉珍指出，過程中雙方互有拉扯、推擠，在野黨立委並沒有做出不法行為，只是基於議事攻防，希望能夠正常開會。陳玉珍表示，民進黨立委針對本應是國會自主的政治議事攻防行為提出傷害告訴，浪費司法資源，請民進黨立委以蒼生為念，讓司法官把時間用來偵辦審理詐騙等

In [8]:
item_id

['aipl_20250312_1',
 'aipl_20250312_2',
 'aipl_20250312_3',
 'aipl_20250312_4',
 'ait_20250312_1',
 'ait_20250312_2',
 'ait_20250312_3',
 'ait_20250311_4',
 'aspt_20250312_1',
 'aspt_20250312_2',
 'aspt_20250312_3',
 'aspt_20250312_4']

# Save data

In [9]:
data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [10]:
df.head(2)

,item_id,date,category,title,content,link,photo_link
0,aipl_20250312_1,2025-03-12,政治,國合會助貝里斯開創香草產業 勉小農婦女經濟自主,國合會今天表示，駐貝里斯技術團開班教種香草，30名小農及婦女日前順利結訓，當地官員期勉提升經...,/news/aipl/202503120373.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
1,aipl_20250312_2,2025-03-12,政治,國會衝突綠委控傷害 藍白立委收到檢方傳票,去年底立法院審查選罷法等案，朝野立委在議場爆發肢體衝突，民進黨立委對國民黨、民眾黨立委提告傷...,/news/aipl/202503120363.aspx,https://imgcdn.cna.com.tw/www/webphotos/WebCov...


In [11]:
df.shape

(12, 7)

In [12]:
df.content[0]

'國合會今天表示，駐貝里斯技術團開班教種香草，30名小農及婦女日前順利結訓，當地官員期勉提升經濟自主；國合會副秘書長謝佩芬則說，香草在貝里斯尚未發展商業化生產，將持續協助當地產業發展。國合會今天發布新聞稿指出，駐貝里斯技術團於6日舉辦「小農香草種植培訓班」結訓典禮，本次培訓為國合會「後疫情時期協助拉丁美洲及加勒比海經濟復甦暨婦女賦權計畫」的重要一環，共有30名學員順利結訓，涵蓋加勒比海農業研究與發展機構（CARDI）人員、農業專業高中師生及社區婦女。國合會指出，課程結束後，每位婦女學員皆獲贈10株香草苗在自家後院實踐所學，開啟小規模生產。來自機構及學校的代表則將帶回香草苗，分別於所屬農場栽種，與兩座區域育苗中心共同發揮示範推廣的作用。駐貝里斯大使徐儷文說，對台灣在貝國香草產業發展初期即提供實質協助感到欣慰。國合會副秘書長謝佩芬表示，香草是昂貴的香料，在食品、化妝品與工業領域需求旺盛，市場前景廣闊，貝里斯具備理想生長環境，卻尚未發展商業化生產，樂見技術團發掘這項契機，未來將持續協助貝里斯投入香草產業發展。'

In [13]:
df.to_csv("cna_category_news.csv", sep="|", index=False)